In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key="your_api_key", transport="rest")

model = genai.GenerativeModel('gemini-2.0-flash')

response = model.generate_content("你是谁？")
print(response.text)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-think

In [2]:
import re
def cleaner(code):
    pat = re.compile(r'(/\*([^*]|(\*+[^*/]))*\*+/)|(//.*)')
    code = re.sub(pat, '', code)
    code = re.sub('\n', '', code)
    code = re.sub('\t', '', code)
    return code

In [ ]:
import pandas as pd
dataset = "reveal"
m1 = pd.read_pickle(f'../../data/finetune/{dataset}/{dataset}_train.pkl')
m2 = pd.read_pickle(f'../../data/finetune/{dataset}/{dataset}_val.pkl')
m3 = pd.read_pickle(f'../../data/finetune/{dataset}/{dataset}_test.pkl')

for df in [m1, m2, m3]:
    if "functionSource" in df.columns:
        df["func"] = df["functionSource"].apply(cleaner)
        
    if dataset == "draper":
        df["target"] = df["combine"] * 1

    if "label" in df.columns:
        df["target"] = df["label"]

    if dataset == "mvd":
        df["target"] = df["target"].apply(lambda x: 1 if x != 0 else 0)

m1 = m1[["func", "target"]]
m2 = m2[["func", "target"]]
m3 = m3[["func", "target"]]

In [ ]:
from openai import OpenAI
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

client = OpenAI(
    api_key="your_api_key",
    base_url="https://api.x.ai/v1",
)

def detect_vulnerability(code):
    try:
        response = client.chat.completions.create(
            model="grok-3-latest",
            messages=[
                {"role": "system", "content": "You are a cybersecurity expert analyzing code for vulnerabilities. Respond with '1' if vulnerable or '0' if safe."},
                {"role": "user", "content": f"Does this code contain security vulnerabilities? Respond with only '1' for yes or '0' for no:\n\n{code}"}
            ],
            stream=False
        )
        return int(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"Error analyzing code: {e}")
        return 0

def calculate_metrics(y_true, y_pred, dataset_name):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"\nMetrics for {dataset_name}:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

def process_dataset_multithread(df: pd.DataFrame, name: str, max_workers: int = 10) -> pd.DataFrame:
    results = [None] * len(df)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {
            executor.submit(detect_vulnerability, code): idx
            for idx, code in enumerate(df['func'])
        }
        for future in tqdm(as_completed(future_to_idx),
                           total=len(future_to_idx),
                           desc=f"Processing {name} set",
                           ncols=80):
            idx = future_to_idx[future]
            try:
                results[idx] = future.result()
            except Exception as e:
                results[idx] = 0

    df['grok_prediction'] = results
    if 'target' in df.columns:
        metrics = calculate_metrics(df['target'], df['grok_prediction'], name)
        df['grok_match'] = (df['grok_prediction'] == df['target']).astype(int)

    return df

def process_datasets_in_parallel(datasets, names, max_workers: int = 10):
    results = {}
    for df, name in zip(datasets, names):
        print(f"\n--- Start {name} set ---")
        results[name] = process_dataset_multithread(df, name, max_workers=max_workers)
    return results

datasets = [m3]
names = ['test']
results = process_datasets_in_parallel(datasets, names, max_workers=8)

results['test'].to_pickle(f'../../data/finetune/{dataset}/{dataset}_test_with_grok.pkl')

for name, df in results.items():
    calculate_metrics(df['target'], df['grok_prediction'], name)


--- Start test set ---


Processing test set:   9%|█▊                 | 213/2274 [00:27<03:07, 10.96it/s]

Error analyzing code: invalid literal for int() with base 10: "there! I've taken a look at the code you provided. After analyzing it, I’ve determined that there are potential security vulnerabilities, particularly related to improper error handling and resource 


Processing test set:  11%|██▏                | 259/2274 [00:33<03:41,  9.12it/s]

Error analyzing code: invalid literal for int() with base 10: "there, I'm diving into this code snippet to check for any security vulnerabilities. After a thorough look, I've determined that this code is safe. It appears to be a straightforward function for diss


...

Processing test set: 100%|█████████████████▉| 2268/2274 [05:36<00:00, 10.72it/s]

Error analyzing code: invalid literal for int() with base 10: "there! I've taken a look at the code snippet you provided, which appears to be related to initializing a DMA (Direct Memory Access) controller for a SPARC32 system in an emulation environment like QE


Processing test set: 100%|██████████████████| 2274/2274 [05:37<00:00,  6.74it/s]


Metrics for test:
Accuracy:  0.2700
Precision: 0.1054
Recall:    0.8304
F1 Score:  0.1871

Metrics for test:
Accuracy:  0.2700
Precision: 0.1054
Recall:    0.8304
F1 Score:  0.1871



/var/folders/zr/c6223m_9071ckmbfh1gxb27r0000gn/T/ipykernel_49664/2922691979.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grok_prediction'] = results
/var/folders/zr/c6223m_9071ckmbfh1gxb27r0000gn/T/ipykernel_49664/2922691979.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grok_match'] = (df['grok_prediction'] == df['target']).astype(int)
